# Retrieve adresses from HERE freemium API

This script retrieves adresses from HERE freemium API (https://engage.here.com/freemium). The API offers up to 250.000 transactions per month, without daily restrictions (as of August 2018), which makes it much more convenient than Gxxgle.
You need to register and get your APP-ID and APP-CODE by retrieving one for JavaScript/REST project. Then simply insert the credentials into geocode_url and off you go!

In [ ]:
import pandas as pd
import time
import requests
import datetime
import logging
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

This is an example dataframe: 

In [ ]:
mydata = pd.DataFrame({'City': ["Berlin", "Paris"], 'Street': ["Pariser Platz", "Square de Berlin"], 'Number': [1, 1], 'Country': ["Germany", "France"]})
mydata['Adress'] = mydata["Street"]+ " " + mydata["Number"].map(str)+", "+ mydata["City"].map(str)+", "+ mydata["Country"].map(str)

You probably have your adresses in a sheet:

In [ ]:
mydata = pd.read_csv(
        "C:/Users/XXX/Data/Adresses.csv", 
        sep=";", engine='python',dtype={'Number': object})
mydata['Adress'] = mydata["Street"]+ " " + mydata["Number"].map(str)+", "+ mydata["City"].map(str)+", COUNTRY"

In [ ]:
mydata.head()

In [ ]:
#remove duplicates to reduce adress list
myadresslist = mydata["Adress"]
myadresslist.shape
myadresslist_wodoubles = list(set(myadresslist))
len(myadresslist_wodoubles)

You need to replace  YOURAPPID  and YOURCODE with your credentials:

In [ ]:
# Specify the column name in your input data that contains addresses here
address_column_name = "Adress"

def get_google_results(address):

    # Set up your Geocoding url
    geocode_url = "https://geocoder.api.here.com/6.2/geocode.json?app_id=YOURAPPID&app_code=YOURCODE-ww&searchtext={}".format(address)
    results = requests.get(geocode_url)
    # Results will be in JSON format - convert to dict using requests functionality
    results = results.json()
    
    # if there's no results or an error, return empty results.
    if len(results['Response']) == 0:
        output = {
            "latitude": None,
            "longitude": None,
        }
    else:    
        answer = results['Response']['View'][0]['Result'][0]['Location']["DisplayPosition"]
        answer
        output = {
        "latitude": answer.get("Latitude"),
        "longitude": answer.get("Longitude"),
        }
        output['input_string'] = address    
    return output

In [ ]:
myadresslist_wodoubles = myadresslist_wodoubles[0:15]
# Create a list to hold results
results = []
BACKOFF_TIME = 2
start = datetime.datetime.now()
print(start)
# Go through each address in turn
for address in myadresslist_wodoubles:
    print(address)
    print(myadresslist_wodoubles.index(address))
    print(start) #so you remember when you started
   
    try:
        geocode_result = get_google_results(address)
        print(geocode_result)
        results.append(geocode_result)
    except:
        print("List does not contain value")
    # Give the API a rest each 50 adresses --> be nice :-)
    if len(results) % 50 == 0:
        time.sleep(BACKOFF_TIME * 60) # sleep for 2 minutes

# All done
logger.info("Finished geocoding all addresses")
end = datetime.datetime.now()
duration = end-start
print(duration) #just out of interest, how long it took
# Write the full results to csv using the pandas library.
pd.DataFrame(results).to_csv('myadresslist.csv',  sep=";", index=False, encoding='utf-16')